<a href="https://colab.research.google.com/github/kingspy89/ml_learnings-/blob/main/mice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Multivariate Imputation by Chained Equations for Missing Value

In [8]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
df = np.round(pd.read_csv('50_Startups.csv')[['R&D Spend','Administration','Marketing Spend','Profit']]/10000)
np.random.seed(9)
df = df.sample(5)
df

,R&D Spend,Administration,Marketing Spend,Profit
21,8.0,15.0,30.0,11.0
37,4.0,5.0,20.0,9.0
2,15.0,10.0,41.0,19.0
14,12.0,16.0,26.0,13.0
44,2.0,15.0,3.0,7.0


In [9]:
df = df.iloc[:,0:-1]
df

,R&D Spend,Administration,Marketing Spend
21,8.0,15.0,30.0
37,4.0,5.0,20.0
2,15.0,10.0,41.0
14,12.0,16.0,26.0
44,2.0,15.0,3.0


In [13]:
df.iloc[1,0] = np.nan
df.iloc[3,1] = np.nan
df.iloc[-1,-1] = np.nan
df.head()

/tmp/ipython-input-1429726703.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[1,0] = np.nan
/tmp/ipython-input-1429726703.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[3,1] = np.nan
/tmp/ipython-input-1429726703.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[-1,-1] = np.nan


,R&D Spend,Administration,Marketing Spend
21,8.0,15.0,30.0
37,NaN,5.0,20.0
2,15.0,10.0,41.0
14,12.0,NaN,26.0
44,2.0,15.0,NaN


In [14]:
df0 = pd.DataFrame()
df0['R&D Spend'] = df['R&D Spend'].fillna(df['R&D Spend'].mean())
df0['Administration']= df['Administration'].fillna(df['Administration'].mean())
df0['marketing Spend'] = df['Marketing Spend'].fillna(df['Marketing Spend'].mean())
df0

,R&D Spend,Administration,marketing Spend
21,8.00,15.00,30.00
37,9.25,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.25,26.00
44,2.00,15.00,29.25


In [1]:
df1 = df0.copy()

df1.iloc[1,0] = np.NaN

df1

NameError: name 'df0' is not defined

In [ ]:

X = df1.iloc[[0,2,3,4],1:3]
X

In [ ]:
y = df1.iloc[[0,2,3,4],0]
y

In [ ]:
lr = LinearRegression()
lr.fit(X,y)
lr.predict(df1.iloc[1,1:].values.reshape(1,2))

In [ ]:
df1.iloc[1,0] = 23.14

In [ ]:

df1.iloc[3,1] = np.NaN

In [ ]:
X = df1.iloc[[0,1,2,4],[0,2]]
X

In [ ]:
y = df1.iloc[[0,1,2,4],1]
y

In [ ]:
lr = LinearRegression()
lr.fit(X,y)
lr.predict(df1.iloc[3,[0,2]].values.reshape(1,2))

In [ ]:
df1.iloc[3,1] = 11.06

In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def manual_mice(df, n_iterations=5, tolerance=1e-4):

    # Work on a copy
    df_imputed = df.copy()

    # Store mask of originally missing values
    missing_mask = df.isna()

    # Step 1: Initial mean imputation
    for col in df_imputed.columns:
        df_imputed[col].fillna(df_imputed[col].mean(), inplace=True)

    # Iterative process
    for iteration in range(n_iterations):

        df_old = df_imputed.copy()

        for col in df.columns:

            # Only process columns that had missing values originally
            if missing_mask[col].sum() == 0:
                continue

            # Define target and features
            y = df_imputed[col]
            X = df_imputed.drop(columns=[col])

            # Rows where original value was NOT missing → training
            train_rows = ~missing_mask[col]
            predict_rows = missing_mask[col]

            # Train model
            model = LinearRegression()
            model.fit(X[train_rows], y[train_rows])

            # Predict missing values
            predicted_values = model.predict(X[predict_rows])

            # Replace values
            df_imputed.loc[predict_rows, col] = predicted_values

        # Convergence check
        diff = np.linalg.norm(df_imputed.values - df_old.values)
        print(f"Iteration {iteration+1}, Difference: {diff}")

        if diff < tolerance:
            print("Converged.")
            break

    return df_imputed

In [16]:
imputed_df = manual_mice(df, n_iterations=10)


Iteration 1, Difference: 14.090496708186553
Iteration 2, Difference: 7.820625585669124
Iteration 3, Difference: 24.166110838322943
Iteration 4, Difference: 7.657361816356142
Iteration 5, Difference: 0.2853059237904406
Iteration 6, Difference: 0.013313536677305787
Iteration 7, Difference: 0.000621683503239911
Iteration 8, Difference: 2.9029952674320815e-05
Converged.


/tmp/ipython-input-60423933.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_imputed[col].fillna(df_imputed[col].mean(), inplace=True)
